<a href="https://colab.research.google.com/github/vincentungu00/Coding-Challenges/blob/main/Restaurant_Orders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime

pd.options.mode.copy_on_write = True

In [2]:
# Load the data sets

menu_items = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/main/Sources/menu_items.csv')
order_details = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/main/Sources/order_details.csv')

In [3]:
menu_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   menu_item_id  32 non-null     int64  
 1   item_name     32 non-null     object 
 2   category      32 non-null     object 
 3   price         32 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ KB


In [4]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12234 entries, 0 to 12233
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_details_id  12234 non-null  int64  
 1   order_id          12234 non-null  int64  
 2   order_date        12234 non-null  object 
 3   order_time        12234 non-null  object 
 4   item_id           12097 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 478.0+ KB


In [5]:
# Process menu_items
menu_items_processed = menu_items.copy().drop_duplicates()

In [6]:
# Process order_details
def get_day(date_str):
  day_of_week = date_str.strftime('%A')
  weekflag = 'Weekend' if day_of_week in ['Saturday', 'Sunday'] else 'Weekday'
  return day_of_week, weekflag

def get_eat_time(hour_str):
  if 6 <= hour_str < 9:
    return 'Breakfast'
  elif 9 <= hour_str < 12:
    return 'Brunch'
  elif 12 <= hour_str < 14:
    return 'Lunch'
  elif 14 <= hour_str < 18:
    return 'Afternoon Snack'
  elif 18 <= hour_str < 20:
    return 'Dinner'
  elif 20 <= hour_str < 23:
    return 'Evening Snack'
  else:
    return 'Late Night'

order_details_deduplicate = order_details.copy().drop_duplicates()
order_details_dropna = order_details_deduplicate.dropna()

order_details_processed = order_details_dropna.groupby(['order_id', 'order_date', 'order_time','item_id'])['item_id'].agg(['count']).rename(columns={'count': 'quantity'}).reset_index()
order_details_processed['order_details_id'] = order_details_processed.groupby(by=['order_id'])['item_id'].cumcount() + 1
order_details_processed.insert(1, 'order_details_id', order_details_processed.pop('order_details_id'))

order_details_processed['order_date'] = pd.to_datetime(order_details_processed['order_date'], format='%m/%d/%y')
order_details_processed['order_date_id'] = order_details_processed['order_date'].dt.strftime('%Y%m%d').astype(int)
order_details_processed['order_date_day'] = order_details_processed['order_date'].dt.day
order_details_processed['order_date_month'] = order_details_processed['order_date'].dt.month
order_details_processed['order_date_year'] = order_details_processed['order_date'].dt.year
order_details_processed['order_date_week'] = order_details_processed['order_date'].dt.isocalendar().week
order_details_processed['order_date_quarter'] = order_details_processed['order_date'].dt.quarter
order_details_processed.insert(6, 'order_date', order_details_processed.pop('order_date'))

order_details_processed['order_time'] = pd.to_datetime(order_details_processed['order_time'], format='%I:%M:%S %p').dt.time
order_details_processed['order_time_id'] = order_details_processed['order_time'].apply(lambda x: x.strftime('%H%M%S')).astype(int)
order_details_processed['order_time_hour'] = order_details_processed['order_time'].apply(lambda x: x.hour)
order_details_processed['order_time_minute'] = order_details_processed['order_time'].apply(lambda x: x.minute)
order_details_processed['order_time_second'] = order_details_processed['order_time'].apply(lambda x: x.second)
order_details_processed['order_time_am_pm'] = order_details_processed['order_time'].apply(lambda x: x.strftime('%p'))

order_details_processed.insert(12, 'order_time', order_details_processed.pop('order_time'))

order_details_processed[['order_date_day_name', 'order_date_weekflag']] = order_details_processed['order_date'].apply(lambda x: get_day(x)).tolist()
order_details_processed['order_eat_time'] = order_details_processed['order_time_hour'].apply(lambda x: get_eat_time(x))

order_details_processed['item_id'] = order_details_processed['item_id'].astype('int64')

del order_details_deduplicate, order_details_dropna

In [7]:
order_details_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11790 entries, 0 to 11789
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   order_id             11790 non-null  int64         
 1   order_details_id     11790 non-null  int64         
 2   item_id              11790 non-null  int64         
 3   quantity             11790 non-null  int64         
 4   order_date_id        11790 non-null  int64         
 5   order_date           11790 non-null  datetime64[ns]
 6   order_date_day       11790 non-null  int32         
 7   order_date_month     11790 non-null  int32         
 8   order_date_year      11790 non-null  int32         
 9   order_date_week      11790 non-null  UInt32        
 10  order_date_quarter   11790 non-null  int32         
 11  order_time_id        11790 non-null  int64         
 12  order_time           11790 non-null  object        
 13  order_time_hour      11790 non-

In [8]:
# Check duplicates on order_date and order_time

order_unique_dates = order_details_processed.groupby('order_id')['order_date'].nunique().sort_values(ascending=False)
print(order_unique_dates)

order_unique_times = order_details_processed.groupby('order_id')['order_time'].nunique().sort_values(ascending=False)
print(order_unique_times)

order_id
1       1
3590    1
3588    1
3587    1
3586    1
       ..
1791    1
1790    1
1789    1
1788    1
5370    1
Name: order_date, Length: 5343, dtype: int64
order_id
1       1
3590    1
3588    1
3587    1
3586    1
       ..
1791    1
1790    1
1789    1
1788    1
5370    1
Name: order_time, Length: 5343, dtype: int64


In [9]:
orders = order_details_processed.merge(menu_items_processed, left_on='item_id', right_on='menu_item_id', how='inner').sort_values(by=['order_id', 'order_details_id']).reset_index(drop=True)
orders['revenue'] = orders['price'].apply(lambda x: x * 0.5)
orders.drop(columns=['item_id'], inplace=True)
orders

,order_id,order_details_id,quantity,order_date_id,order_date,order_date_day,order_date_month,order_date_year,order_date_week,order_date_quarter,...,order_time_second,order_time_am_pm,order_date_day_name,order_date_weekflag,order_eat_time,menu_item_id,item_name,category,price,revenue
0,1,1,1,20230101,2023-01-01,1,1,2023,52,1,...,36,AM,Sunday,Weekend,Brunch,109,Korean Beef Bowl,Asian,17.95,8.975
1,2,1,1,20230101,2023-01-01,1,1,2023,52,1,...,40,AM,Sunday,Weekend,Brunch,106,French Fries,American,7.00,3.500
2,2,2,1,20230101,2023-01-01,1,1,2023,52,1,...,40,AM,Sunday,Weekend,Brunch,108,Tofu Pad Thai,Asian,14.50,7.250
3,2,3,1,20230101,2023-01-01,1,1,2023,52,1,...,40,AM,Sunday,Weekend,Brunch,117,Chicken Burrito,Mexican,12.95,6.475
4,2,4,1,20230101,2023-01-01,1,1,2023,52,1,...,40,AM,Sunday,Weekend,Brunch,124,Spaghetti,Italian,14.50,7.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11785,5369,1,1,20230331,2023-03-31,31,3,2023,13,1,...,4,PM,Friday,Weekday,Evening Snack,109,Korean Beef Bowl,Asian,17.95,8.975
11786,5369,2,1,20230331,2023-03-31,31,3,2023,13,1,...,4,PM,Friday,Weekday,Evening Snack,120,Steak Torta,Mexican,13.95,6.975
11787,5369,3,1,20230331,2023-03-31,31,3,2023,13,1,...,4,PM,Friday,Weekday,Evening Snack,122,Chips & Salsa,Mexican,7.00,3.500
11788,5369,4,1,20230331,2023-03-31,31,3,2023,13,1,...,4,PM,Friday,Weekday,Evening Snack,129,Mushroom Ravioli,Italian,15.50,7.750


In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11790 entries, 0 to 11789
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   order_id             11790 non-null  int64         
 1   order_details_id     11790 non-null  int64         
 2   quantity             11790 non-null  int64         
 3   order_date_id        11790 non-null  int64         
 4   order_date           11790 non-null  datetime64[ns]
 5   order_date_day       11790 non-null  int32         
 6   order_date_month     11790 non-null  int32         
 7   order_date_year      11790 non-null  int32         
 8   order_date_week      11790 non-null  UInt32        
 9   order_date_quarter   11790 non-null  int32         
 10  order_time_id        11790 non-null  int64         
 11  order_time           11790 non-null  object        
 12  order_time_hour      11790 non-null  int64         
 13  order_time_minute    11790 non-

In [13]:
orders.to_csv('orders.csv', index=False)
menu_items_processed.to_csv('menu_items_processed.csv', index=False)
order_details_processed.to_csv('order_details_processed.csv', index=False)

In [14]:
fact_cols = ['order_id', 'order_details_id', 'order_date_id', 'order_time_id', 'menu_item_id', 'quantity']
dim_date_cols = ['order_date_id', 'order_date', 'order_date_day', 'order_date_month', 'order_date_year', 'order_date_week', 'order_date_quarter', 'order_date_day_name', 'order_date_weekflag']
dim_time_cols = ['order_time_id', 'order_time', 'order_time_hour', 'order_time_minute', 'order_time_second', 'order_time_am_pm', 'order_eat_time']
dim_menu_cols = ['menu_item_id', 'item_name', 'category', 'price', 'revenue']

In [15]:
fact_table = orders[fact_cols].drop_duplicates()
dim_date_table = orders[dim_date_cols].drop_duplicates()
dim_time_table = orders[dim_time_cols].drop_duplicates()
dim_menu_table = orders[dim_menu_cols].drop_duplicates()

In [16]:
fact_table.shape, dim_date_table.shape, dim_time_table.shape, dim_menu_table.shape

((11790, 6), (90, 9), (4985, 7), (32, 5))

In [17]:
fact_table.to_csv('fact_table.csv', index=False)
dim_date_table.to_csv('dim_date_table.csv', index=False)
dim_time_table.to_csv('dim_time_table.csv', index=False)
dim_menu_table.to_csv('dim_menu_table.csv', index=False)